## Remove birth years based on the personids in validation data

Put CBDB data1.mdb to 99_compare_model_and_rules folder
and then run the codes


In [1]:
import pyodbc
import pandas as pd

In [2]:
validation_set = pd.read_csv('../validation_set.csv',encoding='utf-8-sig')
validation_set_personid = validation_set['person_id'].tolist()
print(validation_set[:10])

   k_2_by  k_4_by  k_6_by  k_8_by  k_22_by  k_23_by  k_24_by  k_25_by  \
0     NaN     NaN     NaN     NaN      NaN      NaN      NaN      NaN   
1     NaN     NaN     NaN     NaN      NaN      NaN      NaN      NaN   
2     NaN     NaN     NaN     NaN      NaN      NaN      NaN      NaN   
3     NaN     NaN     NaN     NaN      NaN      NaN      NaN      NaN   
4     NaN     NaN     NaN     NaN      NaN      NaN      NaN      NaN   
5     NaN     NaN     NaN     NaN      NaN      NaN      NaN      NaN   
6     NaN     NaN     NaN     NaN      NaN      NaN      NaN      NaN   
7     NaN     NaN     NaN     NaN      NaN      NaN      NaN      NaN   
8     NaN     NaN     NaN     NaN      NaN      NaN      NaN      NaN   
9     NaN     NaN     NaN     NaN      NaN      NaN      NaN      NaN   

   k_26_by  k_28_by  ...  e_323  e_326  e_329  e_330  e_332  e_334  e_343  \
0      NaN      NaN  ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
1      NaN      NaN  ...    NaN    NaN    

In [3]:
import os

def find_mdb_file():
    for root, dirs, files in os.walk("."):
        for file in files:
            if file.endswith(".mdb"):
                return os.path.join(root, file)

mdb_file = find_mdb_file()
print(mdb_file)


.\CBDB_20240208_DATA1.mdb


In [4]:
conn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
            f'DBQ={mdb_file};')

In [5]:
sql = "SELECT * FROM ALTNAME_CODES;"
with pyodbc.connect(conn_str) as conn:
    df = pd.read_sql(sql, conn)
df.head()

,c_name_type_code,c_name_type_desc,c_name_type_desc_chn
0,-1,[Missing Data],[Missing Data]
1,0,Unknown,未詳
2,3,"Alternate Personal Name, Previously Used Name",别名、曾用名
3,4,Courtesy name,字
4,5,"Studio name, Style name",室名、別號


In [6]:
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
batch_size = 50
for i in range(0, len(validation_set_personid), batch_size):
    batch = validation_set_personid[i:i+batch_size]
    query = f"UPDATE BIOG_MAIN SET c_birthyear = NULL WHERE c_personid IN ({', '.join('?' for _ in batch)})"
    cursor.execute(query, batch)
    conn.commit()
cursor.execute(query, batch)
conn.commit()
cursor.close()
conn.close()